In [ ]:
import itertools
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import os
import math

INPUT_FILE = "suburb_info.xlsx"
FEATURES = ["number_of_houses", "number_of_units", "population", "aus_born_perc", "median_income"]
TARGET = "median_house_price"
LAMBDAS = [-1.0, -0.5, 0.0, 0.5, 1.0]
SCALERS = ["minmax", "robust", "zscore"]
CV_FOLDS = 3
RANDOM_STATE = 0

# Load
if not os.path.exists(INPUT_FILE):
    raise FileNotFoundError(f"Could not find {INPUT_FILE} in the working directory. Make sure the file is present.")

df = pd.read_excel(INPUT_FILE)

# Convert 'aus_born_perc' from '67%' → 67.0 (float)
df["aus_born_perc"] = df["aus_born_perc"].astype(str).str.replace('%', '', regex=False)
df["aus_born_perc"] = pd.to_numeric(df["aus_born_perc"], errors="coerce")

# Convert 'median_income' from '$1,583' → 1583 (int)
df["median_income"] = (
    df["median_income"]
    .astype(str)
    .str.replace(r"[\$,]", "", regex=True)
    .astype(float)
)

# Convert 'median_house_price' from '$1,148,100' → 1148100 (int)
df["median_house_price"] = (
    df["median_house_price"]
    .astype(str)
    .str.replace(r"[\$,]", "", regex=True)
    .astype(float)
)

# Check positives (Box-Cox requires > 0)
for col in FEATURES + [TARGET]:
    if (df[col] <= 0).any():
        raise ValueError(f"Column {col} contains non-positive values; Box-Cox requires strictly positive values.")

X_orig = df[FEATURES].astype(float).copy()
y = df[TARGET].astype(float).copy()

results = []
kf = KFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)
model = LinearRegression()

lambda_combos = list(itertools.product(LAMBDAS, repeat=len(FEATURES)))
total_runs = len(SCALERS) * len(lambda_combos)
print(f"Total runs to evaluate: {total_runs}")

for scaler_name in SCALERS:
    for lambdas in lambda_combos:
        # Box-Cox transform each feature with its lambda
        X_trans = np.zeros_like(X_orig.values, dtype=float)
        for i, col in enumerate(FEATURES):
            lam = lambdas[i]
            X_trans[:, i] = stats.boxcox(X_orig.iloc[:, i].values, lmbda=lam)

        # Scale
        if scaler_name == "minmax":
            scaler = MinMaxScaler(feature_range=(0, 1))
        elif scaler_name == "robust":
            scaler = RobustScaler()
        else:
            scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_trans)

        # CV
        r2_scores = cross_val_score(model, X_scaled, y.values, cv=kf, scoring="r2")
        neg_mse_scores = cross_val_score(model, X_scaled, y.values, cv=kf, scoring="neg_mean_squared_error")
        mse_scores = -neg_mse_scores
        rmse_scores = np.sqrt(mse_scores)

        results.append({
            "scaler": scaler_name,
            "lambdas": tuple(lambdas),
            "mean_R2": float(np.mean(r2_scores)),
            "std_R2": float(np.std(r2_scores)),
            "mean_RMSE": float(np.mean(rmse_scores)),
            "std_RMSE": float(np.std(rmse_scores))
        })

res_df = pd.DataFrame(results).sort_values(by="mean_R2", ascending=False).reset_index(drop=True)
out_path = "boxcox_scaling_results.csv"
res_df.to_csv(out_path, index=False)

print("Best result (by mean R^2):")
best = res_df.iloc[0]
print(f"  scaler: {best['scaler']}")
print(f"  lambdas (per feature order {FEATURES}): {best['lambdas']}")
print(f"  mean R^2: {best['mean_R2']:.6f} (std {best['std_R2']:.6f})")
print(f"  mean RMSE: {best['mean_RMSE']:.6f} (std {best['std_RMSE']:.6f})")
print(f"All results saved to: {out_path}")


{'number_of_houses': [np.float64(0.0), np.float64(0.5), np.float64(-0.5)], 'number_of_units': [np.float64(0.0), np.float64(0.5), np.float64(-0.5)], 'population': [np.float64(0.5), np.float64(1.0), np.float64(1.5)], 'aus_born_perc': [np.float64(3.0), np.float64(2.5), np.float64(3.5)], 'median_income': [np.float64(0.5), np.float64(1.0), np.float64(0.0)]}
Total runs to evaluate: 729
Best result (by mean R^2):
  scaler: minmax
  lambdas (per feature order ['number_of_houses', 'number_of_units', 'population', 'aus_born_perc', 'median_income']): (np.float64(0.5), np.float64(0.0), np.float64(1.5), np.float64(2.5), np.float64(1.0))
  mean R^2: 0.666431 (std 0.078910)
  mean RMSE: 270254.737312 (std 37885.187524)
All results saved to: boxcox_scaling_results.csv


In [17]:
print(df)

               suburb  number_of_houses  number_of_units   municipality  \
0          ABBOTSFORD              2304             4706          Yarra   
1          ABERFELDIE              1410              453  Moonee Valley   
2           ALBANVALE              1897              138       Brimbank   
3              ALBION              1389             1392       Brimbank   
4          ALPHINGTON              1729             1099        Darebin   
..                ...               ...              ...            ...   
197  WILLIAMS LANDING              2735              173        Wyndham   
198           WINDSOR              2201             4448    Stonnington   
199           WOLLERT              6516              259     Whittlesea   
200         YALLAMBIE              1286               81        Banyule   
201        YARRAVILLE              5855             2072    Maribyrnong   

     aus_born_perc  median_income  median_house_price  population  
0               68         1797

In [3]:
"""
boxcox_grid_search.py

Run a grid search over:
 - scaling_method: 'minmax' or 'robust' (same for all features)
 - box-cox lambda per feature, chosen from LAMBDAS = [-1, -0.5, 0, 0.5, 1]

Features (input columns expected in suburb_info.xlsx):
    number_of_houses, number_of_units, population, aus_born_perc, median_income

Target:
    median_house_price

Saves results to `boxcox_grid_results.csv` and prints the best config by mean R^2.
"""

import itertools
import math
import time
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
from sklearn.utils import check_random_state
from tqdm import tqdm

# ------------- USER SETTINGS -------------
INPUT_FILE = "suburb_info.xlsx"   # must be in working dir
SHEET_NAME = None                 # or set to sheet name if needed
FEATURE_COLS = [
    "number_of_houses",
    "number_of_units",
    "population",
    "aus_born_perc",
    "median_income",
]
TARGET_COL = "median_house_price"

SCALING_METHODS = ["minmax", "robust"]   # choose one for all features per experiment
LAMBDAS = [-1.0, -0.5, 0.0, 0.5, 1.0]    # box-cox lambdas to choose from (per feature)
CV_FOLDS = 5
RANDOM_STATE = 42
N_JOBS = 1   # set >1 if you want parallelization (not used here, but could be)
OUTPUT_CSV = "boxcox_grid_results.csv"
# ------------- end settings ----------------

rng = check_random_state(RANDOM_STATE)

# ---------- helper functions ----------
def safe_shift_array(x, min_allowed=1e-6):
    """
    Return shifted array and the shift value so that min(x_shifted) > 0.
    We choose shift = (min_allowed - min_x) if min_x <= min_allowed else 0.
    This ensures positivity for Box-Cox.
    """
    min_x = np.min(x)
    if min_x <= min_allowed:
        shift = (min_allowed - min_x)
        return x + shift, shift
    else:
        return x.copy(), 0.0

def boxcox_transform(x, lam):
    """
    Apply Box-Cox style transform with explicit lambda to a positive array x (>0).
    Uses formula:
      if lambda != 0: (x**lambda - 1) / lambda
      else:            log(x)
    Note: we do not standardize after this function; caller can do that.
    """
    if lam == 0.0:
        return np.log(x)
    else:
        return (np.power(x, lam) - 1.0) / lam

def scale_matrix(X_df, method):
    """
    Scale a pandas DataFrame (columns are features) using the same method for all columns.
    method: 'minmax' or 'robust' (as requested by you)
    Returns scaled_df, plus dictionary of parameters used (min,max or median,IQR) for info.
    """
    X = X_df.copy()
    params = {}
    if method == "minmax":
        for c in X.columns:
            xmin = X[c].min()
            xmax = X[c].max()
            if xmax == xmin:
                # constant column -> set to 0.0
                X[c] = 0.0
                params[c] = {"xmin": xmin, "xmax": xmax}
            else:
                X[c] = (X[c] - xmin) / (xmax - xmin)
                params[c] = {"xmin": xmin, "xmax": xmax}
    elif method == "robust":
        # robust scaling as you specified: (x - median) / IQR
        for c in X.columns:
            med = X[c].median()
            q75 = np.percentile(X[c], 75)
            q25 = np.percentile(X[c], 25)
            iqr = q75 - q25
            if iqr == 0:
                # avoid divide by zero
                X[c] = 0.0
                params[c] = {"median": med, "iqr": iqr}
            else:
                X[c] = (X[c] - med) / iqr
                params[c] = {"median": med, "iqr": iqr}
    else:
        raise ValueError("Unknown scaling method: " + str(method))
    return X, params

# ---------- load data ----------
fn = Path(INPUT_FILE)
if not fn.exists():
    raise FileNotFoundError(f"Input file {INPUT_FILE} not found in working directory. Place your suburb_info.xlsx there.")

df = pd.read_excel(fn, sheet_name=SHEET_NAME)

# Basic checks
missing_cols = [c for c in FEATURE_COLS + [TARGET_COL] if c not in df.columns]
if missing_cols:
    raise ValueError(f"Missing required columns in Excel file: {missing_cols}")

X_raw = df[FEATURE_COLS].copy()
y = df[TARGET_COL].values

# Remove rows with missing target or any missing feature rows (or you can impute beforehand)
mask_complete = (~X_raw.isna().any(axis=1)) & (~pd.isna(y))
if mask_complete.sum() != len(df):
    print(f"Warning: dropping {len(df) - mask_complete.sum()} rows with NA in features/target.")
X_raw = X_raw.loc[mask_complete].reset_index(drop=True)
y = y[mask_complete]

# ---------- prepare grid ----------
# we will enumerate lambdas for each feature in the same order as FEATURE_COLS
lambda_combinations = list(itertools.product(LAMBDAS, repeat=len(FEATURE_COLS)))
total_runs = len(SCALING_METHODS) * len(lambda_combinations)
print(f"Total permutations to run: {total_runs} ({len(SCALING_METHODS)} scaling methods * {len(lambda_combinations)} lambda combos)")

# ---------- iterate and evaluate ----------
results = []
kf = KFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_STATE)

# We'll use a progress bar:
for scaling_method in SCALING_METHODS:
    # First scale (same scaling for all features) -> returns DataFrame
    X_scaled, scaling_params = scale_matrix(X_raw, scaling_method)

    # For each lambda-tuple, apply boxcox per column (with shift if needed), then evaluate CV
    # Note: shifting is done using the values after scaling. Shifts are computed per-feature so box-cox input >0.
    for lambdas in tqdm(lambda_combinations, desc=f"Scaling={scaling_method}", leave=False):
        # Copy scaled data for transformation
        X_trans = X_scaled.copy()
        shifts = {}
        # Apply per-column boxcox with specified lambda
        # # Apply Box–Cox first
        # for i, col in enumerate(FEATURE_COLS):
        #     lam = lambdas[i]
        #     X_trans[col] = boxcox_transform(X_raw[col].values, lam)

        # # Then scale once (same method for all 5 transformed columns)
        # X_scaled, _ = scale_matrix(X_trans, scaling_method)

        for i, col in enumerate(FEATURE_COLS):
            lam = lambdas[i]
            arr = X_trans[col].values.astype(float)
            arr_shifted, shift = safe_shift_array(arr, min_allowed=1e-6)
            # keep shift noted so you can backtrack if desired
            shifts[col] = shift
            # apply boxcox formula on arr_shifted
            transformed = boxcox_transform(arr_shifted, lam)
            # If the transformed column has NaNs or infs, handle:
            if np.any(np.isinf(transformed)) or np.any(np.isnan(transformed)):
                # skip this configuration as invalid
                mean_r2 = np.nan
                std_r2 = np.nan
                mean_rmse = np.nan
                std_rmse = np.nan
                results.append({
                    "scaling": scaling_method,
                    "lambdas": tuple(lambdas),
                    "shifts": shifts,
                    "mean_R2": mean_r2,
                    "std_R2": std_r2,
                    "mean_RMSE": mean_rmse,
                    "std_RMSE": std_rmse,
                    "valid": False,
                })
                break
            # replace column
            X_trans[col] = transformed
        else:
            # All columns transformed properly; evaluate linear regression with CV
            model = LinearRegression()
            # cross_val_score for R^2
            r2_scores = cross_val_score(model, X_trans.values, y, cv=kf, scoring="r2")
            # cross_val_score for neg MSE -> convert to RMSE
            neg_mse_scores = cross_val_score(model, X_trans.values, y, cv=kf, scoring="neg_mean_squared_error")
            # convert
            mse_scores = -neg_mse_scores
            rmse_scores = np.sqrt(mse_scores)

            results.append({
                "scaling": scaling_method,
                "lambdas": tuple(lambdas),
                "shifts": shifts,
                "mean_R2": float(np.mean(r2_scores)),
                "std_R2": float(np.std(r2_scores, ddof=1)),
                "mean_RMSE": float(np.mean(rmse_scores)),
                "std_RMSE": float(np.std(rmse_scores, ddof=1)),
                "valid": True,
            })

# ---------- save results ----------
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(["mean_R2"], ascending=False, na_position="last").reset_index(drop=True)
results_df.to_csv(OUTPUT_CSV, index=False)
print(f"Results saved to {OUTPUT_CSV}")

# ---------- print best config ----------
if results_df["valid"].any():
    best_row = results_df.loc[results_df["valid"]].iloc[0]
    print("\nBest configuration by mean R^2:")
    print(f"Scaling method: {best_row['scaling']}")
    print(f"Box-Cox lambdas (in order of features): {best_row['lambdas']}")
    print("Feature order: ", FEATURE_COLS)
    print(f"Mean R^2: {best_row['mean_R2']:.5f} (std {best_row['std_R2']:.5f})")
    print(f"Mean RMSE: {best_row['mean_RMSE']:.5f} (std {best_row['std_RMSE']:.5f})")
else:
    print("No valid configurations found (all failed). Check data for problematic values.")

# Optionally show top N
print("\nTop 5 configs (by mean R^2):")
print(results_df.head(5))


AttributeError: 'dict' object has no attribute 'columns'

I have a data file, suburb_info.xlsx, which contain the following non-zero positive numeric columns: number_of_houses, number_of_units, population, aus_born_perc, median_income, median_house_price. I want to make a linear model to predict median_house_price using the other 5 attributes.  for this, i want the features to be on the same scale, and to have as much linear relationship with median_house_price as possible.  

to do this, i want to create the following framework:  
each individual attribute should be transformed with box-cox power transformation with varying values of lambda (the 5 common values, -1, -0.5, 0, 0.5, 1 will do).  
then, all 5 attributes should be scaled (with the same scale) of either:   
1. min-max scaling to [0, 1] = $$x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}$$   
2. robust scaling: $$x_{scaled} = \frac{x - x_{median}}{IQR(x)}$$   

the framework should run a simple linear model on every permutation of picking one scaling method for all 5 attributes, and then varying lambda values for box-cox transformation for each of the 5 attributes.  
save the RMSE and R^2 metrics for each permutation, and denote which scaling and box-cox lambda parameters yields the best R^2 value.  
to optimize performance, use only 3-fold CV.

I have a data file, suburb_info.xlsx, which contain the following numeric columns: number_of_houses, number_of_units, population, aus_born_perc, median_income, median_house_price. I want to make a linear model to predict median_house_price using the other 5 attributes. for this, i want the features to be on the same scale, and to have as much linear relationship with median_house_price as possible.  
to do this, i want to create the following framework:  
first, all 5 attributes should be scaled (with the same scale) of either: 1. min-max scaling to [0, 1] = $$x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}}$$ 2. robust scaling: $$x_{scaled} = \frac{x - x_{median}}{IQR(x)}$$ then, each individual attribute should be transformed with box-cox power transformation with varying values of lambda (the 5 common values, -1, -0.5, 0, 0.5, 1 will do). the framework should run a simple linear model on every permutation of picking one scaling method for all 5 attributes, and then varying lambda values for box-cox transformation for each of the 5 attributes. save the RMSE and R^2 metrics for each permutation, and denote which scaling and box-cox lambda parameters yields the best R^2 value.